In [1]:
!pip -q install torch_geometric rectools
!pip -q install comet_ml
!pip -q install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 95.7 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.1/727.1 kB 20.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 85.1 MB/s eta 0:00:00:00:01


In [2]:
import comet_ml
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model

from dotenv import load_dotenv
import os

In [ ]:
load_dotenv(".env")

True

In [4]:
experiment = Experiment(
  api_key=os.getenv('API_KEY'),
  project_name="gnn-recommender",
  workspace="annanet",
  log_code=True
)

experiment.set_name('emogat-beauty')
experiment.add_tags(['beauty', 'leave-n-out'])

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/annanet/gnn-recommender/8477a963840245068e0088d3f180ec6b

COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


In [5]:
hyperparameters = {
    'seed': 42,
    'types_of_feedback': ["explicit_positive", "expliсit_negative",
                          "implicit_positive", "implicit_negative"],
    'train_edge_type': ('item','to_feedback_explicit_positive','explicit_positive'),
    'train_num_epochs': 100,
    'train_lr': 3e-4,
    'train_batch_size': 32768,
    'train_print_every': 25,  
    'train_test_every': 25,
    'test_topk': 10,
    'test_batch_size': 32768
}

In [6]:
import os
os.listdir('/kaggle/input/data/leave-n-out/beauty')

['train.csv', 'test.csv']

In [7]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import HeteroData
from torch_geometric.nn import HeteroConv, SAGEConv, GATConv

from sklearn.preprocessing import LabelEncoder

from rectools import Columns
from rectools.metrics import MAP, Precision, Recall, NDCG, calc_metrics

import gc
import random

In [8]:
SEED = hyperparameters['seed']
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

In [9]:
rootpath = '/kaggle/input/data/leave-n-out/beauty/'
train = pd.read_csv(
    rootpath+'train.csv'
)
train['date'] = pd.to_datetime(train['unix_time'])
print(train.head())

                 user_id     item_id  rating  \
0  A00414041RD0BXM6WK0GX  B007IY97U0     3.0   
1  A00414041RD0BXM6WK0GX  B00870XLDS     2.0   
2  A00414041RD0BXM6WK0GX  B008MIRO88     1.0   
3  A00414041RD0BXM6WK0GX  B00BQYYMN0     3.0   
4  A00414041RD0BXM6WK0GX  B00GRTQBTM     5.0   

                                         review_text   unix_time       date  
0  Good quality wig, but the blonde is much more ...  2014-07-14 2014-07-14  
1  Very thin and not as long as the photos :( Aft...  2014-07-14 2014-07-14  
2  Very thin and not as long as the photos :( Aft...  2014-07-14 2014-07-14  
3  This is a great quality wig, however it is a m...  2014-07-14 2014-07-14  
4  This is my absolute favorite wig! I have purch...  2014-07-14 2014-07-14  


In [10]:
explicit_positive = train[(train["rating"] == 5)].index
explisit_negative = train[(train["rating"] <= 2)].index

explicit_combined_feedback = explicit_positive.union(explisit_negative)
print('Количество explicit позитивного фидбека', explicit_positive.shape[0])
print('Количество explicit негативного фидбека', explisit_negative.shape[0])

Количество explicit позитивного фидбека 90800
Количество explicit негативного фидбека 17504


In [11]:
implicit_positive = train[(train["rating"] == 4)].index
implicit_negative = train[(train["rating"] == 3)].index

implicit_combined_feedback = implicit_positive.union(implicit_negative)
print('Количество implicit позитивного фидбека', implicit_positive.shape[0])
print('Количество implicit негативного фидбека', implicit_negative.shape[0])

Количество implicit позитивного фидбека 30668
Количество implicit негативного фидбека 17110


In [12]:
train.loc[:, "target"] = ""
train.loc[explicit_positive, "target"] = "explicit_positive"
train.loc[explisit_negative, "target"] = "expliсit_negative"
train.loc[implicit_positive, "target"] = "implicit_positive"
train.loc[implicit_negative, "target"] = "implicit_negative"

train = train[['user_id','item_id','target','date']]
train.head()

,user_id,item_id,target,date
0,A00414041RD0BXM6WK0GX,B007IY97U0,implicit_negative,2014-07-14
1,A00414041RD0BXM6WK0GX,B00870XLDS,expliсit_negative,2014-07-14
2,A00414041RD0BXM6WK0GX,B008MIRO88,expliсit_negative,2014-07-14
3,A00414041RD0BXM6WK0GX,B00BQYYMN0,implicit_negative,2014-07-14
4,A00414041RD0BXM6WK0GX,B00GRTQBTM,explicit_positive,2014-07-14


In [13]:
train = train.sort_values(by=["user_id", "date"]).reset_index(drop=True)

In [14]:
test = pd.read_csv(
    rootpath+'test.csv'
)
test['date'] = pd.to_datetime(test['unix_time'])
print(test.head())

                 user_id     item_id  rating  \
0  A02155413BVL8D0G7X6DN  B0089JVEPO     5.0   
1  A02155413BVL8D0G7X6DN  B001G2LWDK     5.0   
2  A02155413BVL8D0G7X6DN  B005Z41P28     5.0   
3  A02155413BVL8D0G7X6DN  B0055MYJ0U     5.0   
4  A02155413BVL8D0G7X6DN  B00117CH5M     3.0   

                                         review_text   unix_time       date  
0  leaves my skin clean and smooth. it is creamy ...  2012-10-25 2012-10-25  
1  Works great, smells good, there is a result. I...  2012-12-06 2012-12-06  
2  it works for my hair. smells like almond. made...  2013-01-17 2013-01-17  
3  got this in the mail from China today! holds m...  2013-04-22 2013-04-22  
4  if you like strong smell of honeysuckles and h...  2013-05-01 2013-05-01  


In [15]:
test = test[['user_id','item_id', 'date']]
test.head()

,user_id,item_id,date
0,A02155413BVL8D0G7X6DN,B0089JVEPO,2012-10-25
1,A02155413BVL8D0G7X6DN,B001G2LWDK,2012-12-06
2,A02155413BVL8D0G7X6DN,B005Z41P28,2013-01-17
3,A02155413BVL8D0G7X6DN,B0055MYJ0U,2013-04-22
4,A02155413BVL8D0G7X6DN,B00117CH5M,2013-05-01


# MVP model v2

In [17]:
test = test[(test.user_id.isin(train.user_id)) & (test.item_id.isin(train.item_id))].copy()
test.shape

(42378, 3)

In [18]:
# 2. Преобразование данных - для куарека не особо нужно, но для других - напоминалка
# делаем всегда! чтобы не сломать ничего дальше и чтобы все индексы были от 0 до N без пропусков
user_encoder = LabelEncoder()
video_encoder = LabelEncoder()

train.loc[:, 'user_id'] = user_encoder.fit_transform(train['user_id'])
train.loc[:, 'item_id'] = video_encoder.fit_transform(train['item_id'])

test.loc[:, 'user_id'] = user_encoder.transform(test['user_id'])
test.loc[:, 'item_id'] = video_encoder.transform(test['item_id'])

train['user_id'] = train['user_id'].astype(int)
train['item_id'] = train['item_id'].astype(int)
test['user_id'] = test['user_id'].astype(int)
test['item_id'] = test['item_id'].astype(int)

In [20]:
# т.е. сразу знаем количество и в каких пределах изменяется user_id и video_id
num_videos = train['item_id'].nunique()
num_users = train['user_id'].nunique()

print('Количество уникальных movie_id', num_videos)
print('Количество уникальных user_id', num_users)

Количество уникальных movie_id 12095
Количество уникальных user_id 22363


In [24]:
def prepare_hetero_data(df) -> HeteroData:
    data = HeteroData()
    users = df['user_id'].unique()
    num_users = len(users)
    num_items = df['item_id'].max() + 1
    feedback_types = df['target'].unique().tolist()

    # узлы
    data['user'].node_id = torch.arange(num_users)
    data['item'].node_id = torch.arange(num_items)
    for ft in feedback_types:
        data[ft].node_id = torch.arange(num_users)

    # ребра item<->feedback->user
    for ft in feedback_types:
        mask = df['target'] == ft
        items = torch.LongTensor(df.loc[mask, 'item_id'].values)
        users_idx = torch.LongTensor(df.loc[mask, 'user_id'].values)
        # item -> fb
        data['item', f'to_feedback_{ft}', ft].edge_index = torch.stack([items, users_idx], dim=0)
        # fb -> item
        data[ft, f'feedback_to_item_{ft}', 'item'].edge_index = torch.stack([users_idx, items], dim=0)
        # fb -> user (1:1)
        idx = torch.arange(num_users)
        data[ft, f'to_user_{ft}', 'user'].edge_index = torch.stack([idx, idx], dim=0)

    # user <-> user
    # data['user', 'interacts', 'user'].edge_index = build_full_graph(num_users)
    return data

In [22]:
class HeteroGNN(nn.Module):
    def __init__(self,
                 num_users: int,
                 num_items: int,
                 feedback_types: list,
                 emb_dim: int = 32,
                 hidden_dim: int = 16,
                 heads: int = 2,
                 dropout: float = 0.2):
        super().__init__()
        self.feedback_types = feedback_types
        self.pos_types = ['implicit_positive', 'explicit_positive']
        self.neg_types = ['implicit_negative', 'expliсit_negative']

        self.user_emb = nn.Embedding(num_users, emb_dim)
        self.item_emb = nn.Embedding(num_items, emb_dim)
        self.fb_emb   = nn.ModuleDict({
            ft: nn.Embedding(num_users, emb_dim) for ft in feedback_types
        })

        conv1, conv2 = {}, {}
        for ft in feedback_types:
            # ft -> user
            conv1[(ft, f'to_user_{ft}', 'user')] = GATConv(emb_dim, hidden_dim,
                                                      heads=heads,
                                                      add_self_loops=False)
            conv2[(ft, f'to_user_{ft}', 'user')] = GATConv(hidden_dim*heads, emb_dim,
                                                      heads=1,
                                                      add_self_loops=False)
            # ft -> item
            conv1[(ft, f'feedback_to_item_{ft}', 'item')] = GATConv(emb_dim, hidden_dim,
                                                       heads=heads,
                                                       add_self_loops=False)
            conv2[(ft, f'feedback_to_item_{ft}', 'item')] = GATConv(hidden_dim*heads, emb_dim,
                                                       heads=1,
                                                       add_self_loops=False)

            # item -> ft
            conv1[('item', f'to_feedback_{ft}', ft)] = GATConv(emb_dim, hidden_dim,
                                                       heads=heads,
                                                       add_self_loops=False)
            conv2[('item', f'to_feedback_{ft}', ft)] = GATConv(hidden_dim*heads, emb_dim,
                                                       heads=1,
                                                       add_self_loops=False)

        self.conv1 = HeteroConv(conv1, aggr='mean')
        self.conv2 = HeteroConv(conv2, aggr='mean')

        # LayerNorm и Dropout
        types = ['user', 'item'] + feedback_types
        self.norm1 = nn.ModuleDict({t: nn.LayerNorm(hidden_dim*heads) for t in types})
        self.norm2 = nn.ModuleDict({t: nn.LayerNorm(emb_dim) for t in types})
        self.dropout = nn.Dropout(dropout)

    def forward(self, data):
        x = {
            'user': self.user_emb(data['user'].node_id),
            'item': self.item_emb(data['item'].node_id)
        }
        for ft in self.feedback_types:
            x[ft] = self.fb_emb[ft](data[ft].node_id)

        h1 = self.conv1(x, data.edge_index_dict)
        for t, h in h1.items():
            h1[t] = self.dropout(F.leaky_relu(self.norm1[t](h)))

        h2 = self.conv2(h1, data.edge_index_dict)
        out = {}
        for t, h in h2.items():
            out[t] = self.norm2[t](h)
            
        return out['user']

In [25]:
data = prepare_hetero_data(train)
data

HeteroData(
  user={ node_id=[22363] },
  item={ node_id=[12095] },
  implicit_negative={ node_id=[22363] },
  expliсit_negative={ node_id=[22363] },
  explicit_positive={ node_id=[22363] },
  implicit_positive={ node_id=[22363] },
  (item, to_feedback_implicit_negative, implicit_negative)={ edge_index=[2, 17110] },
  (implicit_negative, feedback_to_item_implicit_negative, item)={ edge_index=[2, 17110] },
  (implicit_negative, to_user_implicit_negative, user)={ edge_index=[2, 22363] },
  (item, to_feedback_expliсit_negative, expliсit_negative)={ edge_index=[2, 17504] },
  (expliсit_negative, feedback_to_item_expliсit_negative, item)={ edge_index=[2, 17504] },
  (expliсit_negative, to_user_expliсit_negative, user)={ edge_index=[2, 22363] },
  (item, to_feedback_explicit_positive, explicit_positive)={ edge_index=[2, 90800] },
  (explicit_positive, feedback_to_item_explicit_positive, item)={ edge_index=[2, 90800] },
  (explicit_positive, to_user_explicit_positive, user)={ edge_index=[2, 2

In [26]:
train.item_id.nunique(), train.item_id.min(), train.item_id.max()

(12095, 0, 12094)

In [27]:
num_users = len(train['user_id'].unique())
num_items = train['item_id'].max() + 1
feedback_types = train['target'].unique().tolist()
model = HeteroGNN(num_users, num_items, feedback_types)

In [28]:
model

HeteroGNN(
  (user_emb): Embedding(22363, 32)
  (item_emb): Embedding(12095, 32)
  (fb_emb): ModuleDict(
    (implicit_negative): Embedding(22363, 32)
    (expliсit_negative): Embedding(22363, 32)
    (explicit_positive): Embedding(22363, 32)
    (implicit_positive): Embedding(22363, 32)
  )
  (conv1): HeteroConv(num_relations=12)
  (conv2): HeteroConv(num_relations=12)
  (norm1): ModuleDict(
    (user): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (item): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (implicit_negative): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (expliсit_negative): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (explicit_positive): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (implicit_positive): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
  )
  (norm2): ModuleDict(
    (user): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (item): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (

In [29]:
model(data)

tensor([[ 0.4926, -0.6532, -1.1236,  ...,  1.2925,  0.8757,  1.8589],
        [-1.1998, -0.1905, -0.3593,  ...,  1.5784, -0.0054,  1.4054],
        [-1.0936, -0.2544, -0.5207,  ...,  0.9713,  0.2837, -0.3997],
        ...,
        [ 0.0887, -0.1390,  0.6247,  ..., -0.1009,  1.1127,  1.2315],
        [ 0.7876,  0.0165, -1.0925,  ...,  1.4862, -0.3878, -1.2298],
        [ 0.1394,  0.1856, -0.8927,  ...,  0.7751, -0.2390,  1.5109]],
       grad_fn=<NativeLayerNormBackward0>)

In [30]:
test_df = test[['user_id', 'item_id']]
interactions = test_df.rename(columns={
    'user_id': Columns.User,
    'item_id': Columns.Item,
})

viewed_items = train.groupby("user_id")["item_id"].agg(set).to_dict()

In [31]:
def train_model(model: HeteroGNN,
                train_data: HeteroData,
                edge_type: tuple,
                num_epochs: int = 10,
                lr: float = 1e-3,
                batch_size: int = 1024,
                device: str = None,
                print_every: int = 100,
                test_every: int = 500,
                top_k: int = 10,
                test_batch_size=2048) -> HeteroGNN:
    device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    train_data = train_data.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    src, dst = train_data[edge_type].edge_index
    num_train = src.size(0)
    test_top_k = top_k * 300
    total_steps = 0
    print(f"Num of training examples: {num_train}")
    for epoch in range(1, num_epochs + 1):
        model.train()
        perm = torch.randperm(num_train, device=device)
        total_loss = 0.0
        step = 0
        for i in range(0, num_train, batch_size):
            idx = perm[i:i + batch_size]
            users = dst[idx]
            pos_items = src[idx]
            neg_items = torch.randint(0, model.item_emb.num_embeddings,
                                      size=pos_items.size(), device=device)
            optimizer.zero_grad()
            user_embs = model(train_data)[users]
            pos_emb = model.item_emb(pos_items)
            neg_emb = model.item_emb(neg_items)
            pos_score = (user_embs * pos_emb).sum(dim=1)
            neg_score = (user_embs * neg_emb).sum(dim=1)
            loss = -torch.log(torch.sigmoid(pos_score - neg_score) + 1e-15).mean()
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * users.size(0)
            step += 1

            experiment.log_metric('Train BPR Loss vs step', loss.item(), step=total_steps)
            if step % print_every == 0 or step == 1:
                avg_loss = total_loss /  (step * batch_size)
                current_lr = optimizer.param_groups[0]['lr']
                d = (pos_score - neg_score).detach().cpu()
                print(f"Epoch {epoch}, Step {step}, LR: {current_lr:.6f}, Current Loss: {loss.item():.4f}, Avg Loss: {avg_loss:.4f}")
                print(f"Diff stats — min: {d.min():.4f}, max: {d.max():.4f}, mean: {d.mean():.4f}, std: {d.std():.4f}")
                print()

                experiment.log_metric('Diff stats (mean) vs step', d.mean(), step=total_steps)
                experiment.log_metric('Diff stats (std) vs step', d.std(), step=total_steps)

            del user_embs, pos_emb, neg_emb, pos_score, neg_score
            gc.collect()
            torch.cuda.empty_cache()
            
            if step % test_every == 0 or step == 1:
                model.eval()
                # model.to('cpu')
                with torch.no_grad():
                    user_emb = model(train_data)
                    item_emb = model.item_emb.weight
                    item_emb_t = item_emb.t().detach()
                    del item_emb
                    gc.collect()

                    scores = []
                    for i in range(0, user_emb.shape[0], test_batch_size):
                        user_e = user_emb[i:i+test_batch_size]
                        rating = torch.mm(user_e, item_emb_t)
                        _, topk_items = torch.topk(rating, k=test_top_k, dim=1)
                        scores.extend(topk_items)

                        del user_e, rating
                        gc.collect()

                    scores = torch.cat(scores, dim=0)
                    scores = scores.reshape((num_users, test_top_k))
                    topk_np = scores.cpu().numpy()

                    filtered_topk = []
                    for u_idx in range(num_users):
                        seen = viewed_items[u_idx]
                        recs = topk_np[u_idx, :]
                        mask = np.isin(recs, list(seen), invert=True)
                        filtered = recs[mask][:top_k] 
                        filtered_topk.append(filtered)
                    
                    users = []
                    items = []
                    ranks = []
                    for u_idx in range(num_users):
                        recs = filtered_topk[u_idx]
                        for rank, item in enumerate(recs[:top_k], 1):
                            users.append(u_idx)
                            items.append(item)
                            ranks.append(rank)
                    
                    reco_df = pd.DataFrame({
                        Columns.User: users,
                        Columns.Item: items,
                        Columns.Rank: ranks,
                    })

                    metrics = {
                        'map@10': MAP(k=10),
                        'precision@10': Precision(k=10),
                        'recall@10': Recall(k=10),
                        'ndcg@10': NDCG(k=10)
                    }
                    
                    results = calc_metrics(
                        metrics=metrics,
                        reco=reco_df,
                        interactions=interactions,
                    )
                print(f"Step/epoch {step}/{epoch}, Test metrics:")
                for key, value in results.items():
                    print(f"{key}: {value:.9f}")
                    experiment.log_metric(f"Test {key} vs step", value, step=total_steps)
                del scores, topk_np
                gc.collect()

                model.to(device)
                model.train()
                train_data.to(device)
            total_steps += 1
        epoch_loss = total_loss / num_train
        experiment.log_metric(f'Train Loss vs epoch', epoch_loss, epoch=epoch - 1)
        print(f"Epoch {epoch} completed, Train Loss: {epoch_loss:.4f}")
    return model

In [32]:
experiment.log_parameters(hyperparameters)

In [33]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [34]:
edge_type = hyperparameters['train_edge_type']
num_epochs = hyperparameters['train_num_epochs']
lr = hyperparameters['train_lr']
batch_size = hyperparameters['train_batch_size']
print_every = hyperparameters['train_print_every']
test_every = hyperparameters['train_test_every']
top_k = hyperparameters['test_topk']
test_batch_size = hyperparameters['test_batch_size']
model = train_model(model,
                    data,
                    edge_type=edge_type,
                    num_epochs=num_epochs,
                    lr=lr,
                    batch_size=batch_size,
                    print_every=print_every,
                    test_every=test_every,
                    top_k=top_k,
                    test_batch_size=test_batch_size)

Num of training examples: 90800
Epoch 1, Step 1, LR: 0.000300, Current Loss: 3.3346, Avg Loss: 3.3346
Diff stats — min: -32.7624, max: 32.3749, mean: -0.1805, std: 7.9348

Step/epoch 1/1, Test metrics:
precision@10: 0.000895804
recall@10: 0.000898423
ndcg@10: 0.000892096
map@10: 0.000266431
Epoch 1 completed, Train Loss: 3.2917
Epoch 2, Step 1, LR: 0.000300, Current Loss: 3.1917, Avg Loss: 3.1917
Diff stats — min: -33.3510, max: 35.8390, mean: 0.1021, std: 7.9210

Step/epoch 1/2, Test metrics:
precision@10: 0.000707214
recall@10: 0.000707214
ndcg@10: 0.000673521
map@10: 0.000188665
Epoch 2 completed, Train Loss: 3.2040
Epoch 3, Step 1, LR: 0.000300, Current Loss: 3.1643, Avg Loss: 3.1643
Diff stats — min: -34.6098, max: 32.8325, mean: 0.1653, std: 7.9210

Step/epoch 1/3, Test metrics:
precision@10: 0.000848656
recall@10: 0.000848656
ndcg@10: 0.000816606
map@10: 0.000228441
Epoch 3 completed, Train Loss: 3.1586
Epoch 4, Step 1, LR: 0.000300, Current Loss: 3.1805, Avg Loss: 3.1805
Diff s

In [35]:
torch.save(model, "gnn_model_mvl.model")
from IPython.display import FileLink

FileLink('gnn_model_mvl.model')

/kaggle/working/gnn_model_mvl.model

In [36]:
# del model
gc.collect()
torch.cuda.empty_cache()

In [37]:
log_model(
    experiment=experiment,
    model=model,
    model_name="GNN",
)

In [38]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : emogat-beauty
COMET INFO:     url                   : https://www.comet.com/annanet/gnn-recommender/8477a963840245068e0088d3f180ec6b
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Diff stats (mean) vs step [100] : (-0.1805461198091507, 4.622474193572998)
COMET INFO:     Diff stats (std) vs step [100]  : (7.139638900756836, 7.965987205505371)
COMET INFO:     Test map@10 vs step [100]       : (0.00018866512501590298, 0.00034544951093748834)
COMET INFO:     Test ndcg@10 vs step [100]      : (0.0006735213679860812, 0.0011579379652054882)
COMET INFO:     Test precision@10 vs step [100] : (0.0007072135785007072, 0.0011551155115511551)
COMET INFO